In [2]:
import numpy as np
import argparse
import cv2

In [3]:
#arguments
args = {
    "image": "Desktop/Screen Shot 2020-07-07 at 12.42.20 PM.png",
    "prototxt": "Desktop/Colorize/Model_Files/colorization_deploy_v2.prototxt",
    "model": "Desktop/Colorize/Model_Files/colorization_release_v2.caffemodel",
    "points": "Desktop/Colorize/Model_Files/pts_in_hull.npy"
}

In [4]:
#Loading the serialized black and white colorizer model and clustering center points from disk
print("Loading model")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])
pts = np.load(args["points"])
#Adding cluster centers as (1x1) convolutions to the loaded model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

Loading model


In [4]:
#Loading the example image, scaling the pixel intensities to the and then converting the image from the BGR to Lab color space
image = cv2.imread(args["image"])
scaled = image.astype("float32") / 255.0
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

In [5]:
#Resizing the Lab image to (224x224), splitting channels, extract the 'L' (Lightness) channel, and performing mean centering
resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

In [6]:
#Passing the L channel through the network which predicts the 'a' (green-red encoder) and 'b' (blue-yellow encoder) channel values
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
#Resizing the predicted 'ab' volume to the same dimensions as the example image
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

In [ ]:
#Grabbing'L' channel from example image (not resized) and concatenating the original 'L' channel with the predicted 'ab' channels
L = cv2.split(lab)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
#Converting the output image from the Lab color space to RGB
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)
colorized = (255 * colorized).astype("uint8")
#Displaying the original and output colorized images
cv2.imshow("Original", image)
cv2.imshow("Colorized", colorized)
cv2.waitKey(0)
cv2.destroyAllWindows()